# Project Part 3

In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn.functional as F
import torch.cuda
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
input_data_path = 'https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv'
df = pd.read_csv(input_data_path)
df.head()

,Text,CB_Label
0,damn there is someones nana up here at beach w...,0
1,no kidding! dick clark was a corpse mechanical...,0
2,i read an article on jobros and thought damn w...,0
3,I got one fucking day of sprinkles and now it'...,0
4,I was already listening to Elliott smith and ...,0


In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [34]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=50 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating Vocabulary for y(dependent variable).  Note that 0 = non-cyberbullying and 1 = cyberbullying

In [35]:
classes = df.CB_Label.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{0: 0, 1: 1}
{0: 0, 1: 1}


Creating a new column with these labels which will be the y that is used

In [36]:
df['label'] = df['CB_Label'].apply(lambda x: class_tok2idx[x])
df.head()

,Text,CB_Label,label
0,damn there is someones nana up here at beach w...,0,0
1,no kidding! dick clark was a corpse mechanical...,0,0
2,i read an article on jobros and thought damn w...,0,0
3,I got one fucking day of sprinkles and now it'...,0,0
4,I was already listening to Elliott smith and ...,0,0


In [37]:

sequence_0 = "Damn  hope you getz better soon !!!"
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)


number of tokens in seq0 is 12
{'input_ids': tensor([[ 101, 8441, 2810, 1128, 1243, 1584, 1618, 1770,  106,  106,  106,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[0.2499, 0.3933, 0.3568]], grad_fn=<SoftmaxBackward0>)

In [38]:
sequence_1 = "Shakespeare nerd!"
seq0_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq1 is 6
{'input_ids': tensor([[  101,  7647, 24928,  2956,   106,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


tensor([[0.2291, 0.4035, 0.3674]], grad_fn=<SoftmaxBackward0>)

In [39]:
sequence_2 = "Is there ever a day that mattresses are not on sale?"
seq0_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq2 is 15
{'input_ids': tensor([[  101,  2181,  1175,  1518,   170,  1285,  1115, 14090,  1279,  1132,
          1136,  1113,  4688,   136,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[0.2452, 0.3978, 0.3570]], grad_fn=<SoftmaxBackward0>)

In [40]:
sequence_3 = "Dood! You just justified your MBA. I pity the sorry ass of that downtrodden employee whose manager you'll soon become."
seq0_tokens = tokenizer(sequence_3, return_tensors="pt")
print(f"number of tokens in seq3 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq3 is 30
{'input_ids': tensor([[  101, 27588,  1181,   106,  1192,  1198, 16489,  1240, 16079,   119,
           146, 13532,  1103,  2959,  3919,  1104,  1115,  1205,  8005, 19221,
          7775,  2133,  2618,  1128,   112,  1325,  1770,  1561,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}


tensor([[0.2250, 0.4285, 0.3465]], grad_fn=<SoftmaxBackward0>)

In [43]:
ds_raw = Dataset.from_pandas(df[['label', 'Text']])

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [45]:
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

In [46]:
ds[0]

{'label': 0,
 'Text': 'damn there is someones nana up here at beach with one. dont think ic an steal and get to you quickly tho ;(',
 'input_ids': [101,
  4477,
  1175,
  1110,
  1800,
  1116,
  9468,
  1605,
  1146,
  1303,
  1120,
  4640,
  1114,
  1141,
  119,
  1274,
  1204,
  1341,
  178,
  1665,
  1126,
  8991,
  1105,
  1243,
  1106,
  1128,
  1976,
  24438,
  1186,
  132,
  113,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [47]:
ds = ds.shuffle(seed=42)
ds[0]

{'label': 0,
 'Text': "haha it was great... i hate how the audio lags so badly though... it'd be more fun in real time lol",
 'input_ids': [101,
  5871,
  2328,
  1122,
  1108,
  1632,
  119,
  119,
  119,
  178,
  4819,
  1293,
  1103,
  6056,
  2495,
  5700,
  1177,
  6118,
  1463,
  119,
  119,
  119,
  1122,
  112,
  173,
  1129,
  1167,
  4106,
  1107,
  1842,
  1159,
  25338,
  1233,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [49]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [50]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 9435
len(ds_eval) = 1665


In [51]:
import os 
os.environ["WANDB_DISABLED"] = "true"

In [52]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="/kaggle/working",
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [54]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else:
    device = "cpu"

Using GPU


In [55]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [56]:
torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy
200,No log,0.469086,0.750150
400,No log,0.451230,0.780180
600,0.465500,0.464765,0.761562
800,0.465500,0.477640,0.764565
1000,0.327100,0.577115,0.756156
1200,0.327100,0.605474,0.751351
1400,0.327100,0.647367,0.748949
1600,0.210500,0.734697,0.752553
1800,0.210500,0.776457,0.760360
2000,0.119100,0.873614,0.749550


{'eval_loss': 1.105977177619934,
 'eval_accuracy': 0.745945945945946,
 'eval_runtime': 2.933,
 'eval_samples_per_second': 567.681,
 'eval_steps_per_second': 18.07,
 'epoch': 10.0}

In [57]:
df_test = pd.read_csv("https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv")
df_test.head()

,Text,CB_Label
0,damn there is someones nana up here at beach w...,0
1,no kidding! dick clark was a corpse mechanical...,0
2,i read an article on jobros and thought damn w...,0
3,I got one fucking day of sprinkles and now it'...,0
4,I was already listening to Elliott smith and ...,0


In [58]:
ds_test_raw = Dataset.from_pandas(df_test)
ds_test_raw[0]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


{'Text': 'damn there is someones nana up here at beach with one. dont think ic an steal and get to you quickly tho ;(',
 'CB_Label': 0}

In [59]:
ds_test = ds_test_raw.map(tokenize_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

In [60]:
preds = trainer.predict(test_dataset=ds_test)

In [61]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions)

In [62]:
df_test['preds'] = test_preds.tolist()
df_test['CB_Label'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()

,Text,CB_Label,preds
0,damn there is someones nana up here at beach w...,0,0
1,no kidding! dick clark was a corpse mechanical...,0,0
2,i read an article on jobros and thought damn w...,0,0
3,I got one fucking day of sprinkles and now it'...,0,0
4,I was already listening to Elliott smith and ...,0,0
